In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import rubin_jupyter_utils.lab.notebook as nb
nb.utils.get_node()

Patching auth into notebook.base.handlers.IPythonHandler(notebook.base.handlers.AuthenticatedHandler) -> IPythonHandler(jupyterhub.singleuser.mixins.HubAuthenticatedHandler, notebook.base.handlers.AuthenticatedHandler)


'andes05.cp.lsst.org'

In [3]:
import os
import sys
import asyncio
import logging

import pandas as pd

from matplotlib import pyplot as plt

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel import MTCS, ComCam
from lsst.ts.observatory.control import RotType

Update leap second table
current_tai uses the system TAI clock


In [4]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)

In [5]:
log = logging.getLogger("setup")
log.level = logging.DEBUG

In [6]:
domain = salobj.Domain()

In [7]:
mtcs = MTCS(domain=domain, log=log)
mtcs.set_rem_loglevel(40)

mtmount: Adding all resources.
mtptg: Adding all resources.
mtaos: Adding all resources.
mtm1m3: Adding all resources.
mtm2: Adding all resources.
mthexapod_1: Adding all resources.
mthexapod_2: Adding all resources.
mtrotator: Adding all resources.
mtdome: Adding all resources.
mtdometrajectory: Adding all resources.


In [8]:
await mtcs.start_task

Read historical data in 0.10 sec
electrical DDS read queue is full (100 elements); data may be lost
elevation DDS read queue is full (100 elements); data may be lost
application DDS read queue is full (100 elements); data may be lost
cameraCableWrap DDS read queue is full (100 elements); data may be lost
actuators DDS read queue is full (100 elements); data may be lost
azimuth DDS read queue is full (100 elements); data may be lost
Read historical data in 0.21 sec
zenithAngle DDS read queue is full (100 elements); data may be lost
electrical DDS read queue is filling: 87 of 100 elements
rotation DDS read queue is full (100 elements); data may be lost
temperature DDS read queue is full (100 elements); data may be lost
timeAndDate DDS read queue is full (100 elements); data may be lost
application DDS read queue is filling: 88 of 100 elements
motors DDS read queue is full (100 elements); data may be lost


[None, None, None, None, None, None, None, None, None, None]

tangentForce DDS read queue is full (100 elements); data may be lost
powerSupplyData DDS read queue is full (100 elements); data may be lost
mountStatus DDS read queue is full (100 elements); data may be lost
actuators DDS read queue is filling: 89 of 100 elements
electrical DDS read queue is full (100 elements); data may be lost
tangentEncoderPositions DDS read queue is full (100 elements); data may be lost
mountPosition DDS read queue is full (100 elements); data may be lost
pidData DDS read queue is full (100 elements); data may be lost
ccwFollowingError DDS read queue is full (100 elements); data may be lost
tangentActuatorSteps DDS read queue is full (100 elements); data may be lost
currentTargetStatus DDS read queue is full (100 elements); data may be lost
inclinometerData DDS read queue is full (100 elements); data may be lost
powerStatus DDS read queue is full (100 elements); data may be lost
imsData DDS read queue is full (100 elements); data may be lost
positionIMS DDS read q

In [9]:
comcam = ComCam(domain=domain, log=log)

cccamera: Adding all resources.
ccheaderservice: Adding all resources.
ccarchiver: Adding all resources.


In [10]:
comcam.set_rem_loglevel(40)

In [11]:
await comcam.start_task

[None, None, None]

In [12]:
target = await mtcs.find_target(el=80, az=0, mag_limit=8)
print(target)

HD  96276


In [13]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky)

Slewing to HD  96276: 11 05 50.7332 -20 24 32.680
Setting rotator physical position to 0.0 deg. Rotator will track sky.
Camera cable wrap following disabled in MTMount.
Stop tracking.
Sending slew command.
Scheduling check coroutines
process as completed...
Monitor position started.
Waiting for Target event from mtmount.
mtmount: <State.ENABLED: 2>
mtptg: <State.ENABLED: 2>
mtaos: <State.ENABLED: 2>
mtm1m3: <State.ENABLED: 2>
mtm2: <State.ENABLED: 2>
mthexapod_1: <State.ENABLED: 2>
mthexapod_2: <State.ENABLED: 2>
mtrotator: <State.ENABLED: 2>
mtdome: <State.ENABLED: 2>
mtdometrajectory: <State.ENABLED: 2>
Wait for mtmount in position event.
MTMount axesInPosition got: elevation True, azimuth False.
MTMount axesInPosition got: elevation False, azimuth False.
Wait for dome in position event.
Wait for rotator in position event.
Mount target: private_revCode: 261ad639, private_sndStamp: 1631808445.120743, private_rcvStamp: 1631808445.1214619, private_seqNum: 2, private_identity: MTMount, p

In [14]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=-1.5)

Slewing to HD  96276: 11 05 50.7332 -20 24 32.680
Setting rotator physical position to -1.5 deg. Rotator will track sky.
Camera cable wrap following disabled in MTMount.
Stop tracking.
Sending slew command.
Command to track target HD  96276 rejected: Rejected : command not allowed in current state


AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1572535570, ack=<SalRetCode.CMD_FAILED: -302>, error=6612, result='Rejected : command not allowed in current state')

In [15]:
# Bringing MTPtg back from FAULT to ENABLE
#  MTPtg has another level of protection for the Rotator. It will be set to FAULT
#  if the difference between the CCW and the Rotator is larger than 2 deg. 
await mtcs.enable()

Enabling all components
Gathering settings.
Couldn't get settingVersions event. Using empty settings.
Complete settings for mtmount.
Complete settings for mtptg.
Complete settings for mtaos.
Complete settings for mtm1m3.
Complete settings for mtm2.
Complete settings for mthexapod_1.
Complete settings for mthexapod_2.
Complete settings for mtrotator.
Complete settings for mtdome.
Complete settings for mtdometrajectory.
Settings versions: {'mtmount': '', 'mtptg': '', 'mtaos': 'comcam', 'mtm1m3': '', 'mtm2': '', 'mthexapod_1': 'default', 'mthexapod_2': 'default', 'mtrotator': '', 'mtdome': '', 'mtdometrajectory': ''}
[mtmount]::[<State.ENABLED: 2>]
[mtptg]::[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[mtaos]::[<State.ENABLED: 2>]
[mtm1m3]::[<State.ENABLED: 2>]
[mtm2]::[<State.ENABLED: 2>]
[mthexapod_1]::[<State.ENABLED: 2>]
[mthexapod_2]::[<State.ENABLED: 2>]
[mtrotator]::[<State.ENABLED: 2>]
[mtdome]::[<State.ENABLED: 2>]
[mtdometrajectory]::[<State.EN

In [17]:
target = await mtcs.find_target(el=80, az=120, mag_limit=8)
print(target)

HD 103332


In [18]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=-1.5)

Slewing to HD 103332: 11 53 49.4709 -34 25 09.997
Setting rotator physical position to -1.5 deg. Rotator will track sky.
Camera cable wrap following disabled in MTMount.
Stop tracking.
Sending slew command.
Command to track target HD 103332 rejected: Rejected : Rotator out of range. Target in rotator limit (low limit:-2 high limit:2 degs) but out of slew limit margin (1 degs)


AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1572535571, ack=<SalRetCode.CMD_FAILED: -302>, error=6611, result='Rejected : Rotator out of range. Target in rotator limit (low limit:-2 high limit:2 degs) but out of slew limit margin (1 degs)')

In [19]:
# Setting MTMtg to STANDBY so Tiago can change the slew limits which should allows
#  us moving to rot=-1.5
await mtcs.set_state(salobj.State.STANDBY, components=["mtptg"])

[mtptg]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
All components in <State.STANDBY: 5>.


In [20]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtptg"])

[mtptg]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
All components in <State.ENABLED: 2>.


In [21]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=-1.5)

Slewing to HD 103332: 11 53 49.4709 -34 25 09.997
Setting rotator physical position to -1.5 deg. Rotator will track sky.
Camera cable wrap following disabled in MTMount.
Stop tracking.
Sending slew command.
Scheduling check coroutines
process as completed...
Monitor position started.
Waiting for Target event from mtmount.
mtmount: <State.ENABLED: 2>
mtptg: <State.ENABLED: 2>
mtaos: <State.ENABLED: 2>
mtm1m3: <State.ENABLED: 2>
mtm2: <State.ENABLED: 2>
mthexapod_1 not in <State.ENABLED: 2>: <State.FAULT: 3>
mthexapod_2: <State.ENABLED: 2>
mtrotator: <State.ENABLED: 2>
mtdome: <State.ENABLED: 2>
mtdometrajectory: <State.ENABLED: 2>
Wait for mtmount in position event.
Wait for dome in position event.
Wait for rotator in position event.


RuntimeError: mthexapod_1 state is <State.FAULT: 3>, expected <State.ENABLED: 2>

In [23]:
await mtcs.stop_tracking()

Stop tracking.


AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=2070072687, ack=<SalRetCode.CMD_FAILED: -302>, error=6612, result='Rejected : command not allowed in current state')

In [24]:
await mtcs.set_state(
    state=salobj.State.ENABLED,
    settings=dict(mthexapod_1="default"),
    components=["mthexapod_1"]
    )

Unable to transition mthexapod_1 to <State.ENABLED: 2> NoneType: None
.
Traceback (most recent call last):
  File "/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-0.7.0/lib/python3.8/site-packages/lsst/ts/salobj/csc_utils.py", line 148, in set_summary_state
    await cmd.start(timeout=timeout)
  File "/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-0.7.0/lib/python3.8/site-packages/lsst/ts/salobj/topics/remote_command.py", line 449, in start
    return await cmd_info.next_ackcmd(timeout=timeout)
  File "/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-0.7.0/lib/python3.8/site-packages/lsst/ts/salobj/topics/remote_command.py", line 182, in next_ackcmd
    raise base.AckError(msg="Command failed", ackcmd=ackcmd)
lsst.ts.salobj.base.AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1874305938, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: You must use the clearError command or the engineering use

RuntimeError: Failed to transition ['mthexapod_1'] to <State.ENABLED: 2>.

In [25]:
await mtcs.rem.mthexapod_1.cmd_clearError.set_start()

In [26]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtptg"])

[mtptg]::[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
All components in <State.ENABLED: 2>.


In [28]:
await mtcs.set_state(
    state=salobj.State.ENABLED,
    settings=dict(mthexapod_1="default"),
    components=["mthexapod_1"]
    )

[mthexapod_1]::[<State.DISABLED: 1>, <State.ENABLED: 2>]
All components in <State.ENABLED: 2>.


In [29]:
target = await mtcs.find_target(el=60, az=120, mag_limit=8)
print(target)

HD 129621


In [30]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky)

Slewing to HD 129621: 14 44 48.3370 -41 02 08.407
Setting rotator physical position to 0.0 deg. Rotator will track sky.
Camera cable wrap following disabled in MTMount.
Stop tracking.
Sending slew command.
Scheduling check coroutines
process as completed...
Monitor position started.
Waiting for Target event from mtmount.
mtmount: <State.ENABLED: 2>
mtptg: <State.ENABLED: 2>
mtaos: <State.ENABLED: 2>
mtm1m3: <State.ENABLED: 2>
mtm2: <State.ENABLED: 2>
mthexapod_1: <State.ENABLED: 2>
mthexapod_2: <State.ENABLED: 2>
mtrotator: <State.ENABLED: 2>
mtdome: <State.ENABLED: 2>
mtdometrajectory: <State.ENABLED: 2>
Wait for mtmount in position event.
Wait for dome in position event.
Wait for rotator in position event.
Mount target: private_revCode: 261ad639, private_sndStamp: 1631813366.8252888, private_rcvStamp: 1631813366.8259535, private_seqNum: 2719, private_identity: MTMount, private_origin: 14416, private_host: 0, elevation: 60.40573773521628, elevationVelocity: 0.0030930130076826795, azim

RuntimeError: mtrotator state is <State.FAULT: 3>, expected <State.ENABLED: 2>

In [31]:
# await mtcs.stop_tracking()

In [32]:
await mtcs.rem.mtrotator.cmd_clearError.set_start()

In [33]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtrotator"])

[mtrotator]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
All components in <State.ENABLED: 2>.


In [34]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtptg"])

[mtptg]::[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
All components in <State.ENABLED: 2>.


In [35]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky)

Slewing to HD 129621: 14 44 48.3370 -41 02 08.407
Setting rotator physical position to 0.0 deg. Rotator will track sky.
Camera cable wrap following disabled in MTMount.
Stop tracking.
Sending slew command.
Scheduling check coroutines
process as completed...
Monitor position started.
Waiting for Target event from mtmount.
mtmount: <State.ENABLED: 2>
mtptg: <State.ENABLED: 2>
mtaos: <State.ENABLED: 2>
mtm1m3: <State.ENABLED: 2>
mtm2: <State.ENABLED: 2>
mthexapod_1: <State.ENABLED: 2>
mthexapod_2: <State.ENABLED: 2>
mtrotator: <State.ENABLED: 2>
mtdome: <State.ENABLED: 2>
mtdometrajectory: <State.ENABLED: 2>
Wait for mtmount in position event.
Wait for dome in position event.
Wait for rotator in position event.
Mount target: private_revCode: 261ad639, private_sndStamp: 1631813566.0652115, private_rcvStamp: 1631813566.0658102, private_seqNum: 2722, private_identity: MTMount, private_origin: 14416, private_host: 0, elevation: 61.02121814886229, elevationVelocity: 0.0030851056628672883, azim

In [36]:
await mtcs.stop_tracking()

Stop tracking.


In [37]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=1.9)

Slewing to HD 129621: 14 44 48.3370 -41 02 08.407
Setting rotator physical position to 1.9 deg. Rotator will track sky.
Camera cable wrap following disabled in MTMount.
Stop tracking.
Sending slew command.
Scheduling check coroutines
process as completed...
Monitor position started.
Waiting for Target event from mtmount.
mtmount: <State.ENABLED: 2>
mtptg: <State.ENABLED: 2>
mtaos: <State.ENABLED: 2>
mtm1m3: <State.ENABLED: 2>
mtm2: <State.ENABLED: 2>
mthexapod_1: <State.ENABLED: 2>
mthexapod_2: <State.ENABLED: 2>
mtrotator: <State.ENABLED: 2>
mtdome: <State.ENABLED: 2>
mtdometrajectory: <State.ENABLED: 2>
Wait for mtmount in position event.
Wait for dome in position event.
Wait for rotator in position event.
Mount target: private_revCode: 261ad639, private_sndStamp: 1631813624.1356933, private_rcvStamp: 1631813624.136383, private_seqNum: 3161, private_identity: MTMount, private_origin: 14416, private_host: 0, elevation: 61.20029840790253, elevationVelocity: 0.0030826571418026963, azimu

RuntimeError: mtrotator state is <State.FAULT: 3>, expected <State.ENABLED: 2>

In [38]:
await mtcs.rem.mtrotator.cmd_clearError.set_start()

In [39]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtrotator"])

[mtrotator]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
All components in <State.ENABLED: 2>.


In [40]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtptg"])

[mtptg]::[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
All components in <State.ENABLED: 2>.


In [41]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=1.5)

Slewing to HD 129621: 14 44 48.3370 -41 02 08.407
Setting rotator physical position to 1.5 deg. Rotator will track sky.
Camera cable wrap following disabled in MTMount.
Stop tracking.
Sending slew command.
Scheduling check coroutines
process as completed...
Monitor position started.
Waiting for Target event from mtmount.
mtmount: <State.ENABLED: 2>
mtptg: <State.ENABLED: 2>
mtaos: <State.ENABLED: 2>
mtm1m3: <State.ENABLED: 2>
mtm2: <State.ENABLED: 2>
mthexapod_1: <State.ENABLED: 2>
mthexapod_2: <State.ENABLED: 2>
mtrotator: <State.ENABLED: 2>
mtdome: <State.ENABLED: 2>
mtdometrajectory: <State.ENABLED: 2>
Wait for mtmount in position event.
Wait for dome in position event.
Wait for rotator in position event.
Mount target: private_revCode: 261ad639, private_sndStamp: 1631813743.9859066, private_rcvStamp: 1631813743.9866495, private_seqNum: 3163, private_identity: MTMount, private_origin: 14416, private_host: 0, elevation: 61.56943989060992, elevationVelocity: 0.0030773885750304568, azim

In [42]:
await comcam.take_object(1)

Generating group_id
imagetype: OBJECT, TCS synchronization not configured.
OBJECT 0001 - 0001


array([2021091600022])

In [43]:
await mtcs.stop_tracking()

Stop tracking.


In [44]:
await mtcs.slew_object(target, rot_type=RotType.Physical, rot=0)

Slewing to HD 129621: 14 44 48.3370 -41 02 08.407
Setting rotator to physical fixed position 0.0 deg. Rotator will not track.
Camera cable wrap following disabled in MTMount.
Stop tracking.
Sending slew command.
Scheduling check coroutines
process as completed...
Monitor position started.
Waiting for Target event from mtmount.
mtmount: <State.ENABLED: 2>
mtptg: <State.ENABLED: 2>
mtaos: <State.ENABLED: 2>
mtm1m3: <State.ENABLED: 2>
mtm2: <State.ENABLED: 2>
mthexapod_1: <State.ENABLED: 2>
mthexapod_2: <State.ENABLED: 2>
mtrotator: <State.ENABLED: 2>
mtdome: <State.ENABLED: 2>
mtdometrajectory: <State.ENABLED: 2>
Wait for mtmount in position event.
Wait for dome in position event.
Wait for rotator in position event.
Mount target: private_revCode: 261ad639, private_sndStamp: 1631814201.844129, private_rcvStamp: 1631814201.8448322, private_seqNum: 4334, private_identity: MTMount, private_origin: 14416, private_host: 0, elevation: 62.973365752121055, elevationVelocity: 0.0030543607217820923

RuntimeError: mtrotator state is <State.FAULT: 3>, expected <State.ENABLED: 2>

In [45]:
await mtcs.rem.mtrotator.cmd_clearError.set_start()
await mtcs.set_state(salobj.State.ENABLED, components=["mtrotator"])

[mtrotator]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
All components in <State.ENABLED: 2>.


In [46]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtptg"])

[mtptg]::[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
All components in <State.ENABLED: 2>.


In [47]:
await mtcs.slew_object(target, rot_type=RotType.Physical, rot=0)

Slewing to HD 129621: 14 44 48.3370 -41 02 08.407
Setting rotator to physical fixed position 0.0 deg. Rotator will not track.
Camera cable wrap following disabled in MTMount.
Stop tracking.
Sending slew command.
Scheduling check coroutines
process as completed...
Monitor position started.
Waiting for Target event from mtmount.
mtmount: <State.ENABLED: 2>
mtptg: <State.ENABLED: 2>
mtaos: <State.ENABLED: 2>
mtm1m3: <State.ENABLED: 2>
mtm2: <State.ENABLED: 2>
mthexapod_1: <State.ENABLED: 2>
mthexapod_2: <State.ENABLED: 2>
mtrotator: <State.ENABLED: 2>
mtdome: <State.ENABLED: 2>
mtdometrajectory: <State.ENABLED: 2>
Wait for mtmount in position event.
Wait for dome in position event.
Wait for rotator in position event.
Mount target: private_revCode: 261ad639, private_sndStamp: 1631814364.9407475, private_rcvStamp: 1631814364.9415314, private_seqNum: 4336, private_identity: MTMount, private_origin: 14416, private_host: 0, elevation: 63.470764632399494, elevationVelocity: 0.003044936282575988

In [49]:
await comcam.get_available_filters()

['i_06', 'r_03', 'g_07']

In [51]:
await comcam.setup_filter('r_03')

Filter r_03 in position.


In [53]:
await mtcs.rem.mthexapod_1.cmd_clearError.set_start()

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=172607481, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: Rejected: initial state is <State.DISABLED: 1> instead of <State.FAULT: 3>')

In [54]:
await mtcs.set_state(
    state=salobj.State.ENABLED,
    settings=dict(mthexapod_1="default"),
    components=["mthexapod_1"]
    )

[mthexapod_1]::[<State.DISABLED: 1>, <State.ENABLED: 2>]
All components in <State.ENABLED: 2>.


In [55]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=1.5)

Slewing to HD 129621: 14 44 48.3370 -41 02 08.407
Setting rotator physical position to 1.5 deg. Rotator will track sky.
Camera cable wrap following disabled in MTMount.
Stop tracking.
Sending slew command.
Scheduling check coroutines
process as completed...
Monitor position started.
Waiting for Target event from mtmount.
mtmount: <State.ENABLED: 2>
mtptg: <State.ENABLED: 2>
mtaos: <State.ENABLED: 2>
mtm1m3: <State.ENABLED: 2>
mtm2: <State.ENABLED: 2>
mthexapod_1: <State.ENABLED: 2>
mthexapod_2: <State.ENABLED: 2>
mtrotator: <State.ENABLED: 2>
mtdome: <State.ENABLED: 2>
mtdometrajectory: <State.ENABLED: 2>
Wait for mtmount in position event.
Wait for dome in position event.
Wait for rotator in position event.
mtrotator not in <State.ENABLED: 2>: <State.FAULT: 3>
Got False
Rotator not in position


RuntimeError: mtrotator state is <State.FAULT: 3>, expected <State.ENABLED: 2>

In [56]:
await mtcs.rem.mtrotator.cmd_clearError.set_start()

In [57]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtrotator"])

[mtrotator]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
All components in <State.ENABLED: 2>.


In [58]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtptg"])

[mtptg]::[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
All components in <State.ENABLED: 2>.


In [59]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=1.5)

Slewing to HD 129621: 14 44 48.3370 -41 02 08.407
Setting rotator physical position to 1.5 deg. Rotator will track sky.
Camera cable wrap following disabled in MTMount.
Stop tracking.
Sending slew command.
Scheduling check coroutines
process as completed...
Monitor position started.
Waiting for Target event from mtmount.
mtmount: <State.ENABLED: 2>
mtptg: <State.ENABLED: 2>
mtaos: <State.ENABLED: 2>
mtm1m3: <State.ENABLED: 2>
mtm2: <State.ENABLED: 2>
mthexapod_1: <State.ENABLED: 2>
mthexapod_2: <State.ENABLED: 2>
mtrotator: <State.ENABLED: 2>
mtdome: <State.ENABLED: 2>
mtdometrajectory: <State.ENABLED: 2>
Wait for mtmount in position event.
Wait for dome in position event.
Wait for rotator in position event.
Got False
Rotator not in position
Mount target: private_revCode: 261ad639, private_sndStamp: 1631814845.422697, private_rcvStamp: 1631814845.4234934, private_seqNum: 11602, private_identity: MTMount, private_origin: 14416, private_host: 0, elevation: 64.92636428241761, elevationVel

In [60]:
await comcam.take_object(15)

Generating group_id
imagetype: OBJECT, TCS synchronization not configured.
OBJECT 0001 - 0001


array([2021091600023])

In [61]:
await mtcs.stop_tracking()

Stop tracking.


In [62]:
await comcam.standby()

[cccamera]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
[ccheaderservice]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
[ccarchiver]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
All components in <State.STANDBY: 5>.
